In [56]:
# Copyright 2019 The Kubeflow Authors
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import kfp
import kfp.dsl as dsl
from kfp import components
import kubernetes as k8s
from kfp import compiler, dsl
from kubernetes.client import V1VolumeMount
import pprint
import numpy as np


@dsl.pipeline(
    name="blueberry detection",
    description="blueberry detection pipeline"
)
def full_pipeline(size: str="2Gi"):
    
    vop = dsl.PipelineVolume(
        pvc="biosens-agrovision-data-volume",
        name="biosens-agrovision-data-volume",
    )

    download_component = kfp.components.load_component_from_file('download/download_model.yaml')
    train_model = kfp.components.load_component_from_file('train/train_model.yaml')
    postprocessing_model = kfp.components.load_component_from_file('postprocessing/postprocessing_model.yaml')
    
# #     #build pipeline
#     download_component_task = download_component(
#         "blueberry-data",
#         "/mnt"
#         ).add_pvolumes({"/mnt": vop})
#     train_model_task = train_model(
#         [1e-1, 1e-2, 1e-3, 1e-4, 1e-5], 
#         [2,4,6], 
#         [5,10,15],
#         [20,40,60,80,100],
#         ['bce', 'kldiv', 'gaussian'],
#         [8,16,32,64],
#         "/mnt/FullSet/trening_set_mini2/img",
#         "/mnt/FullSet/validation_set_mini2/img",
#         "/mnt/FullSet/test_set_mini2/img",
#         download_component_task.output).add_pvolumes({"/mnt": vop})
#     postprocessing_model = postprocessing_model("/mnt/logs4/Train_BGFG_BCE_with_weightsUnet3/",
#                                                 train_model_task.output
#                                                ).add_pvolumes({"/mnt": vop})
    #build pipeline
    download_component_task = download_component(
        "blueberry-data",
        "/mnt"
        ).add_pvolumes({"/mnt": vop})
    train_model_task = train_model(
        [1e-1], 
        [2], 
        [5],
        [20],
        ['bce'],
        [8],
        "/mnt/FullSet/trening_set_mini2/img",
        "/mnt/FullSet/validation_set_mini2/img",
        "/mnt/FullSet/test_set_mini2/img",
        download_component_task.output).add_pvolumes({"/mnt": vop})
    postprocessing_model = postprocessing_model(
        "/mnt/logs4/Train_BGFG_BCE_with_weightsUnet3/",
        "blueberry-results",
        train_model_task.output
        ).add_pvolumes({"/mnt": vop})    

    
    
    
    print("gotovo sve")
if __name__ == '__main__':
    file_name = "full_pipeline.yaml"
    kfp.compiler.Compiler().compile(full_pipeline, file_name)

gotovo sve


In [58]:
import kfp
import random
client=kfp.Client()
filepath = '/home/jovyan/final_working/' + file_name
name = 'blueberry_full_pipeline_sastanak_' + str(random.randint(30000,90000))
print("Uploaded pipeline:", name)
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').


Uploaded pipeline: blueberry_full_pipeline_sastanak_74611
